In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

from data.data_loader import TimeSeriesDataLoader
import pandas as pd
import numpy as np
from layers.multi_cp import SPCI_and_EnbPI
from models.lstm import LSTMModel

print("Data loader imported successfully!")

Data loader imported successfully!


In [2]:
data_loader = TimeSeriesDataLoader()
train_loader, val_loader, test_loader, dates, scaler = data_loader.create_dataloaders(
    frequency='weekly',
    lookback=12,
    forecast_horizon=1,
    train_end_date='2015-12-31',
    val_end_date='2016-12-31',
    test_end_date='2027-12-31',
    batch_size=32
)

Loaded data: (5032, 50) (2005-01-03 00:00:00 ~ 2024-12-30 00:00:00)
Resampled to weekly: (1044, 50)
Preprocessed (fit up to 2015-12-31): (1044, 50)
Created sequences - X: (1032, 12, 50), y: (1032, 1, 50)

Data split by date:
Train: 561 samples (2005-04-01 00:00:00 ~ 2015-12-25 00:00:00)
Val:   53 samples (2016-01-01 00:00:00 ~ 2016-12-30 00:00:00)
Test:  418 samples (2017-01-06 00:00:00 ~ 2025-01-03 00:00:00)


TypeError: TimeSeriesDataset.__init__() got an unexpected keyword argument 'inverse_fn'

In [ ]:
X_train, Y_train = train_loader.dataset.X, train_loader.dataset.y
X_valid, Y_valid = val_loader.dataset.X, val_loader.dataset.y
X_predict, Y_predict = test_loader.dataset.X, test_loader.dataset.y


conformal_predictor = SPCI_and_EnbPI(
    X_train, X_valid, X_predict,
    Y_train, Y_valid, Y_predict,
    model_cls=LSTMModel, 
    scaler=scaler  # <-- Pass the scaler here
)

In [ ]:
X_valid.size()

torch.Size([53, 12, 50])

In [ ]:
Y_predict.size()

torch.Size([418, 1, 50])

In [ ]:
results = conformal_predictor.fit_bootstrap_models_online_multistep(B=3, EPOCHS=10) # B is the number of bootstrap models
conformal_predictor.compute_Widths_Ensemble_online(alpha=0.05, smallT=False, use_SPCI=True) # alpha is your significance level
conformal_predictor.get_results()
# 4. Call the new method to get results in the original scale
# original_scale_results = conformal_predictor.()

# # You can now access the results
# if original_scale_results:
#     original_predictions = original_scale_results['predictions']
#     original_lower_bounds = original_scale_results['lower_bounds']
#     original_upper_bounds = original_scale_results['upper_bounds']
    
#     print("\nSample of inverse-transformed results:")
#     print("Predictions:", original_predictions[0])
#     print("Lower Bounds:", original_lower_bounds[0])
#     print("Upper Bounds:", original_upper_bounds[0])

has inverse on train? False
has inverse on valid? False
has inverse on test? False
Model Num 0 (LSTMModel), Epoch 1, Train Loss: 1.0713, Valid Loss: 3.1564
Model Num 0 (LSTMModel), Epoch 2, Train Loss: 0.7300, Valid Loss: 2.6306
Model Num 0 (LSTMModel), Epoch 3, Train Loss: 0.5594, Valid Loss: 2.1516
Model Num 0 (LSTMModel), Epoch 4, Train Loss: 0.4531, Valid Loss: 1.7716
Model Num 0 (LSTMModel), Epoch 5, Train Loss: 0.3645, Valid Loss: 1.4605
Model Num 0 (LSTMModel), Epoch 6, Train Loss: 0.3065, Valid Loss: 1.1993
Model Num 0 (LSTMModel), Epoch 7, Train Loss: 0.2603, Valid Loss: 1.0134
Model Num 0 (LSTMModel), Epoch 8, Train Loss: 0.2262, Valid Loss: 0.8748
Model Num 0 (LSTMModel), Epoch 9, Train Loss: 0.2070, Valid Loss: 0.7681
Model Num 0 (LSTMModel), Epoch 10, Train Loss: 0.1851, Valid Loss: 0.6887
Loading best model for LSTMModel_0 with validation loss: 0.6887
------------------------------------------------------------
Model Num 1 (LSTMModel), Epoch 1, Train Loss: 1.0613, Valid L

ValueError: window shape cannot be larger than input array shape